<table>
  <tr>
      <td><div align="left"><font size="20" >Central camera projection model</font></div></td>
     <td><img src="images/RVSS-logo.png" width="400"></td>
  </tr>
</table>

In [ ]:
%matplotlib notebook

import sys
!{sys.executable} -m pip install machinevision-toolbox-python

import numpy as np
import math
from spatialmath import SE3
from spatialmath.base import e2h, h2e
import machinevisiontoolbox as mvtb

# From first principles

Let's define some parameters of our camera

In [ ]:
f = 8*1e-3     # focal length in metres
rho = 10*1e-6  # pixel side length in metres
u0 = 500       # principal point, horizontal coordinate
v0 = 500       # principal point, vertical coordinate

Next we define some matrices.  

The first is a 3x3 _intrinsic_ matrix that contains information about the camera itself:
  - focal length of the lens
  - pixel size
  - the position of the principal point in units of pixels,this is where the _principal ray_ pierces the image plane.

In [ ]:
K = np.array([  [f/rho, 0, u0], 
                [0, f/rho, v0], 
                [0, 0, 1]])

The next is a 3x4 matrix that performs the projection, reduces the 3D world point to a 2D image plane point

In [ ]:
P0 = np.array([ [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

Then finally a 4x4 _extrinsic_ matrix_ which says how the camera is positioned and oriented in space. An identity matrix has our camera at the world frame origin and looking along the world z-axis.

In [ ]:
X = np.identity(4)

We multiply these three matrices together to form the camera matrix

In [ ]:
C = K @ P0 @ X

The resulting camera matrix is a 3x4 matrix

In [ ]:
C.shape

Next we define a point in terms of its 3D coordinates (in units of metres), as a column vector

In [ ]:
P = np.array([1, -2, 5])

To perform the projection we need to convert our 3D coordinate to homogeneous form, which is done by the helper function `e2h`

In [ ]:
e2h(P)

Now we can premultiply this by the camera matrix to get the image plane coordinates in homoogeneous form

In [ ]:
C @ e2h(P)

and we use the helper function `h2e` to convert that back to Euclidean coordinates - the 2D image plane coordinates in units of pixels

In [ ]:
h2e( C @ e2h(P) )

If we move the point 0.5m in the x-axis direction we see that the u-coordinate in the image plane increases, as we would expect

In [ ]:
P1 = np.array([3.5, 4, 5])
h2e( C @ e2h(P1) )

Now we will move the camera 0.5m in the x-axis direction. We do this by setting the top right element of the _extrinsic_ matrix so as to represent a translation of 0.5m.

Then we recompute the _camera_ matrix and project our original point onto the image plane of the moved camera.  We see that the u-coordinate has decreased as we would expect - when we move our head to the right the world appears to move to the left.

In [ ]:
X[0,0] = 0.5
C = K @ P0 @ X
h2e( C @ e2h(P) )

# Using the Toolbox CentralCamera class



In [ ]:
camera = mvtb.CentralCamera(f=f, rho=rho, pp=(u0, v0), imagesize=1000)
print(camera)

We can very conveniently project points to the image plane

In [ ]:
# appears to be bug in MVTB toolbox cannot pass in a (3,) or (3,1) array for camera.project
# so we will simply create a copy of the point to make it a (3,2)
P_tmp = np.repeat(P.reshape((3,1)), 2, axis=1) 
camera.project(P_tmp)

and we can plot it on a virtual image plane conveniently by

In [ ]:
camera.plot(P_tmp)

We can project the same point, but this time with the camera moved 0.2m in the x-direction

In [ ]:
camera.project(P_tmp, pose=SE3(0.6, 0, 0))

and we see that the u-coordinate has decreased.  If we look out along the camera's principal axis then moving the camera to the right causes the image plane point to move to the left.

We can extract the intrinsic matrix

In [ ]:
camera.K

and the camera matrix

In [ ]:
camera.C